In [1]:
import pandas as pd

df = pd.read_csv("enedis_clean.csv")

assert "adresse_clean" in df.columns

# Table des adresses uniques à géocoder
adresses_unique = (
    df[["adresse_clean"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

print(len(adresses_unique), "adresses uniques à géocoder")
adresses_unique.head()

482090 adresses uniques à géocoder


,adresse_clean
0,52 RUE ETIENNE DOLET BONDY
1,2 PLACE EUDEMON BONDY
2,211 AVENUE GALLIENI BONDY
3,132 AVENUE GALLIENI BONDY
4,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE BONDY


In [2]:
import requests
import time

def geocode_ban(adresse: str, pause: float = 0.0):
    """
    Géocodage via addok en local (http://localhost:7878/search).
    Compatible avec le format JSON de l'API adresse.data.gouv.fr.
    """
    if not isinstance(adresse, str) or adresse.strip() == "":
        return {
            "adresse_clean": adresse,
            "adress_ban": None,
            "ban_lat": None,
            "ban_lon": None,
            "ban_score": None,
            "ban_id": None,
        }

    # API locale
    url = "http://localhost:7878/search"
    params = {
        "q": adresse,
        "limit": 1
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        # Même logique que pour l'API officielle : on suppose "features"
        features = data.get("features", [])
        if len(features) == 0:
            res = {
                "adress_ban": None,
                "ban_lat": None,
                "ban_lon": None,
                "ban_score": None,
                "ban_id": None,
            }
        else:
            feat = features[0]
            props = feat.get("properties", {})
            geometry = feat.get("geometry", {})

            coords = geometry.get("coordinates", [None, None])
            lon = coords[0]
            lat = coords[1]

            res = {
                "adress_ban": props.get("label"),
                "ban_lat": lat,
                "ban_lon": lon,
                "ban_score": props.get("score"),
                "ban_id": props.get("id"),
            }

    except Exception as e:
        print(f"Erreur BAN locale pour '{adresse}': {e}")
        res = {
            "adress_ban": None,
            "ban_lat": None,
            "ban_lon": None,
            "ban_score": None,
            "ban_id": None,
        }

    # Pas vraiment besoin de pause en local, mais on garde l’option
    if pause and pause > 0:
        time.sleep(pause)

    res["adresse_clean"] = adresse
    return res

In [3]:
# Échantillon de 1000 lignes
df_sample = df.sample(n=1000, random_state=42).copy()

results = []
for i, adresse in enumerate(df_sample["adresse_clean"]):
    res = geocode_ban(adresse)  # plus de pause obligatoire
    results.append(res)

    if (i + 1) % 100 == 0:
        print(f"{i + 1} adresses traitées...")

ban_sample = pd.DataFrame(results)
ban_sample.head()

100 adresses traitées...
200 adresses traitées...
300 adresses traitées...
400 adresses traitées...
500 adresses traitées...
600 adresses traitées...
700 adresses traitées...
800 adresses traitées...
900 adresses traitées...
1000 adresses traitées...


,adress_ban,ban_lat,ban_lon,ban_score,ban_id,adresse_clean
0,10 Rue du tunnel 64700 Hendaye,43.356741,-1.771562,0.960313,64260_2200_00010,10 RUE DU TUNNEL HENDAYE
1,12 Rue Cécile Vallet 92340 Bourg-la-Reine,48.785048,2.315348,0.963415,92014_1440_00012,12 RUE CECILE VALLET BOURG LA REINE
2,42 Cours Fauriel 42100 Saint-Étienne,45.428667,4.397697,0.781338,42218_3390_00042,42 CRS FAURIEL SAINT ETIENNE
3,11 Rue Jean Ferrat 95480 Pierrelaye,49.027482,2.151829,0.960117,95488_0261_00011,11 RUE JEAN FERRAT PIERRELAYE
4,4 Rue Hélène Boucher 31700 Cornebarrieu,43.644979,1.318573,0.952579,31150_0074_00004,4 RUE HELENE BOUCHER CORNEBARRIEU


In [4]:
from tqdm import tqdm
import os

output_file = "ban_enedis_full.csv"
batch_size = 10_000

results = []
first_write = not os.path.exists(output_file)

n = len(adresses_unique)
print("Nombre d'adresses à géocoder :", n)

for i, addr in enumerate(tqdm(adresses_unique["adresse_clean"], total=n, desc="Géocodage BAN local")):
    res = geocode_ban(addr)   # pas de pause
    results.append(res)

    if (i + 1) % batch_size == 0:
        chunk_df = pd.DataFrame(results)

        mode = "w" if first_write else "a"
        header = first_write

        chunk_df.to_csv(output_file, index=False, mode=mode, header=header)
        print(f"Chunk écrit : {i+1} lignes")

        results = []
        first_write = False

# Flush final
if len(results) > 0:
    chunk_df = pd.DataFrame(results)
    mode = "w" if first_write else "a"
    header = first_write

    chunk_df.to_csv(output_file, index=False, mode=mode, header=header)
    print(f"Dernier chunk écrit : {n} lignes au total")


Nombre d'adresses à géocoder : 482090


Géocodage BAN local:   2%|██▍                                                                                                                   | 10048/482090 [00:25<19:45, 398.15it/s]

Chunk écrit : 10000 lignes


Géocodage BAN local:   4%|████▉                                                                                                                 | 20048/482090 [00:50<17:24, 442.27it/s]

Chunk écrit : 20000 lignes


...

Chunk écrit : 470000 lignes


Géocodage BAN local: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 480066/482090 [20:24<00:05, 365.90it/s]

Chunk écrit : 480000 lignes


Géocodage BAN local: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 482090/482090 [20:30<00:00, 391.81it/s]

Dernier chunk écrit : 482090 lignes au total
